In [1]:
from tokenizers import Tokenizer
from pathlib import Path
import sys

sys.path.append(str(Path(Path.cwd()).parent))

from helper.sentence_destructor import destruct_sentence

ModuleNotFoundError: No module named 'tokenizers'

### Test dataset destruction

In [2]:

tokenizer: Tokenizer = Tokenizer.from_file("../../config/input_tokenizer.json")

text = "مەن مەكتەپتىن قايتىپ كەلدىم، ئۇمۇ مەكتەپتىن قايتىل كەلدى، بىز ھەممىمىز مەكتەپتىن قايتىپ كەلدۇق!"

for i in range(10):
    destructed = destruct_sentence(tokenizer, text, 0.15)

    print(destructed)
print(text)

مەنامەكبپەپت<r قايتىنپ كەلدىم، ئۇمۇ مەكاتەپتفىن tقايتىلز كەلدى، بىز ھممىمىز مەكتjتىن قاي)تىپ كەلدۇق!
مەن مڭكتە›پتى قايتى كەلىم، ئۇمۇ مەكتەپتىن ق1يتىلqكەلدى، بىز ھەممشمىز مەكتەپتىن قايتىپ كەلدcدۇ<قس
مەن مەكتەپتى قايتىپ كەدىمف، ئ9ۇمۇ8مەكتەپتىن قيتوىل6كtلدى، بىز ھ=ممىمىز مەlتپتىن قايتىپ كە دۇd!
مەن مەتەپتىن.قايتىپ$ەلدىم، ئۇمۇ مە8كتەتىن قايتىل كەلدى0 بىزھەممىمىز مەتەپتىنc قايتىپ كەلدۇe!
!ەن مكkەپتىن قايتىp كەwلدى5g،ئۇۇ مەكتەپتىن قايتىل-ەلدى، بىز ھەمم$مىز مكتەپتىن 7ايتىپ كەلدۇق!
مەن مەكتەپتىن قايتىپ كەلدىم،t ئۇمۇ مەكەپتىن قايتىq كە8لفى،tgىز ھەممnىامىزtمەكتلپتىن dقايتىپ كەلدۇق!
من مەكتە-تىن قايتىپ كەلدىم، ئۇمۇ مەكتەپت4ن ق۔يت"ىل كەلدى5،بىزھەمم?مىز› ەكتەپتىن قايتپ كەلدۇ۔ق!
م7ن مەكتەپتىن پقايتىپ كەلدىم، ۇمشۇ [ەكتەپتىن قا^يتىj %كەلدى، بىز ھممىمىز مە*تەپتى[قغيتىپ كەلدۇق!
مە~ مەكتەپmىqن قاي£تىپ كەلدىم،]ۇمۇ مەتەپىن قايتىل كەلدى، بىز ھەممىمىoز مەكتەپۆىنقايتىپ كەلدlۇق_!
مەن مە«كتەپت؛تىن قايتىپكەلدىم، ئۇمۇ مەكتەتىن قايتىل كەلدى، بچsز ھەممىمgز مەكە[تىن قا«تپ كە#د=ق!
مەن مەكتەپتىن قايتىپ كەلدىم، ئۇمۇ مە

### Shirnk sentence length by split sentences as sub-sentences by splitting their boundaries using dot (.)

In [7]:
import json

data_dir = Path(Path.cwd()).parent.parent / ".data"
dataset_file = data_dir / "text" / "2000.json"
with open(dataset_file, "r") as file:
    sentences: list[str] = json.loads(file.read())
print(f"sentence count before split: {len(sentences)}")
expanded_sentences: list[str] = []

for s in sentences:
    ss = [ss for ss in s.strip().split(".") if not (ss.strip() == "")]
    ss = [s for s in ss if len(s) > 15]
    expanded_sentences.extend(ss)
print(f"sentence count after split: {len(expanded_sentences)}")

max_sentence_length = 0
for s in expanded_sentences:
    max_sentence_length = max(max_sentence_length, len(s))

print(f"max sentence length after split: {max_sentence_length}")

sentence count before split: 12503
sentence count after split: 22262
max sentence length after split: 1019


### Write the shirnked data to a temporary working file.

In [4]:
with open(data_dir / "dataset.json", "w+") as file:
    file.write(json.dumps(expanded_sentences, ensure_ascii=False))
print("[OK] of write shirnked sentence to file.")

[OK] of write shirnked sentence to file.


### ChatGPT says we can minimize memory re-allocation overhead by pre-allocate fixed sized buffer on the GPU and copy the data into gpu before model call in training loop. lets do some benchmark to validate this.

In [5]:
import torch
import time

my_device = "cuda:0"

dummy_data1 = torch.randint(-32768, 32767, (1024, 2048, 128), dtype=torch.int32, device=my_device)
dummy_data2 = torch.randint(-32768, 32767,(1024, 2048, 128), dtype=torch.int32, device=my_device)

my_buffer = torch.zeros((1024, 2048, 128), dtype=torch.int32, device=my_device)

begin = time.time()
for i in range(100):
    my_buffer[:, :, :] = dummy_data1[:, :, :]
    my_buffer[:, :, :] = dummy_data2[:, :, :]
elapsed_ms = int((time.time() - begin) * 100)

del dummy_data1, dummy_data2, my_buffer
torch.cuda.empty_cache()

print(f"time to taken by copy data to buffer 2000 times: {elapsed_ms}")

time to taken by copy data to buffer 2000 times: 0


### Let's see how much vram taken by all the sentences if we move them into GPU entirely.

In [6]:
import torch
training_data = torch.zeros(len(expanded_sentences), max_sentence_length, dtype=torch.int32, device="cuda:0")